<a href="https://colab.research.google.com/github/kwojtylko/flat_price_estimator/blob/main/3_model_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import bibliotek
import pandas as pd
import numpy as np

In [6]:
# wczytywanie danych
gratka_df = pd.read_csv("gratka_dane_model.csv")
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2,district
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14399332,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",44.93,2,7.0,10700,Krowodrza
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Podgórze, małopolskie","Nowe mieszkanie Kraków Podgórze, ul. Golikówka",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-podgorze-ul-golikowka/ob/19878365,Kukla Nieruchomości prezentujeMieszkanie 2- pokojowe o powierzchni 42 m2 zlokalizowane w domu wolnostojącym przy ulicy Golikówka w dzielnicy Rybitwy. Mieszkanie,42.00,2,1.0,7024,Podgórze
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14403264,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",59.15,3,1.0,9400,Krowodrza
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398368,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",50.56,2,7.0,10300,Krowodrza
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398296,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",76.35,4,6.0,9700,Krowodrza
...,...,...,...,...,...,...,...,...,...,...,...,...
3476,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790039,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,42.22,2,0.0,9995,Płaszów
3477,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790031,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,67.07,3,1.0,8916,Płaszów
3478,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790023,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na p

In [10]:
#zdefiniownie cech do modelu
model_col = ["area_m2", "n_rooms", "n_floor", "district", "price_per_m2"]
model_df = gratka_df[model_col]
model_df

,area_m2,n_rooms,n_floor,district,price_per_m2
0,44.93,2,7.0,Krowodrza,10700
1,42.00,2,1.0,Podgórze,7024
2,59.15,3,1.0,Krowodrza,9400
3,50.56,2,7.0,Krowodrza,10300
4,76.35,4,6.0,Krowodrza,9700
...,...,...,...,...,...
3476,42.22,2,0.0,Płaszów,9995
3477,67.07,3,1.0,Płaszów,8916
3478,63.05,3,1.0,Płaszów,9056
3479,62.59,3,0.0,Płaszów,9394


In [11]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3481 entries, 0 to 3480
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_m2       3481 non-null   float64
 1   n_rooms       3481 non-null   int64  
 2   n_floor       3481 non-null   float64
 3   district      2918 non-null   object 
 4   price_per_m2  3481 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 136.1+ KB


In [12]:
model_df.describe()

,area_m2,n_rooms,n_floor,price_per_m2
count,3481.000000,3481.000000,3481.000000,3481.000000
mean,55.448914,2.559609,2.392129,10631.935363
std,26.350922,1.027479,2.197552,2941.337028
min,13.300000,1.000000,0.000000,4999.000000
25%,39.000000,2.000000,1.000000,8893.000000
50%,51.080000,3.000000,2.000000,9940.000000
75%,66.110000,3.000000,4.000000,11955.000000
max,230.000000,6.000000,14.000000,40000.000000
